### RAG Application
1. Set up a vector store
2. Embed and add documents
3. Query documents
4. Create prompt for LLM
5. Generate response from LLM

In [2]:
import os
import configparser
from langchain_groq import ChatGroq
from langchain_ai21 import AI21Embeddings
from langchain_core.vectorstores import InMemoryVectorStore


# set up config parser
config = configparser.ConfigParser()
config.read("../config.ini")  # holds secrets and keys


# load Groq config
groq = config["groq"]
os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')

# select the model.
model = ChatGroq(model="llama3-8b-8192")

# load AI21 config
ai21 = config['ai21']
os.environ['AI21_API_KEY'] = ai21.get('AI21_API_KEY')



print('setup done successfully')


setup done successfully


### Embeddings
1. We are using AI21 for embeddings. https://python.langchain.com/api_reference/ai21/embeddings/langchain_ai21.embeddings.AI21Embeddings.html#langchain_ai21.embeddings.AI21Embeddings


In [3]:

embeddings = AI21Embeddings()

# Generate embedding for a single document.
text = 'Hello I think LangChain rocks!'
vector = embeddings.embed_query(text)
print('Single Document: ',len(vector), vector)

# Multiple documents
text = [
    '''Emily, an intrepid explorer, embarked on a solo adventure in her reliable Toyota Corolla. As she drove, the familiar roads transformed into a canvas of discovery. Each turn unveiled hidden gems, from tranquil lakes to cozy cafes. 
    The journey became a metaphor for life, teaching Emily that the unknown often holds the most precious treasures. With every mile, she felt a deeper connection to herself and the world around her.
    Her spontaneous road trip was a reminder that sometimes the greatest rewards lie in the path we dare to take, leading to a richer, more fulfilling life.''',
    
    '''In a lush green meadow, a curious rabbit named Hoppy ventured out on a sunny morning. With a twinkle in his eye, he hopped towards a mysterious forest, intrigued by the unknown.
    As he ventured deeper, Hoppy discovered a magical world filled with colorful flowers and friendly creatures. He made new friends, from wise old owls to playful squirrels.
    His adventure taught him about courage, friendship, and the beauty of nature. With a heart full of joy, Hoppy returned home, sharing his tales of wonder with his fellow rabbits, inspiring them to explore the world beyond their burrows. '''
    
    '''John, a blind man, navigated the world with an unwavering spirit. Despite his physical challenge, he embraced life with open arms. Guided by his trusty cane and the warmth of his smile, he ventured into the city, a place of endless possibilities.
    As he walked, he felt the energy of the bustling streets, the laughter of children, and the melody of street musicians. John's heart was filled with the love and support of his family, who were his guiding light.
    His journey taught him that true vision lies within, and he inspired others with his resilience and positive outlook on life.'''
]
vector = embeddings.embed_documents(text)
print('Multiple Documents: ',len(vector), vector)






Single Document:  1024 [0.009869611822068691, -0.010493033565580845, -0.017919501289725304, -0.033225834369659424, 0.03642647713422775, -0.014665713533759117, 0.009937127120792866, 0.03712629899382591, 0.026856884360313416, -0.027925387024879456, 0.017253413796424866, 0.023650452494621277, -0.05084574595093727, -0.020679382607340813, -0.03033445216715336, -0.023262247443199158, -0.007082357071340084, 0.03164554014801979, 0.00044642749708145857, -0.0023669349029660225, 0.044439647346735, -0.014183505438268185, -0.005691415164619684, -0.03202352672815323, -0.01572163961827755, -0.019813349470496178, -0.03331081569194794, -0.058945268392562866, 0.0016432260163128376, -0.06029008328914642, -0.005386717617511749, 0.013347168453037739, -0.023859413340687752, -0.013223502784967422, -0.040835876017808914, 0.00882594846189022, 0.06523624807596207, 0.04417570307850838, -0.031405579298734665, 0.019862396642565727, -0.019811050966382027, 0.055336762219667435, -0.016595810651779175, 0.0079693449661

### Vector Store - InMemoryVectorStore
1. We are using LangChain provided one: https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.in_memory.InMemoryVectorStore.html#langchain_core.vectorstores.in_memory.InMemoryVectorStore
2. Using cosine similarity using numpy to get results.
3. Searching with score results a structure of type: `list[tuple[Document, float]]`
4. Searching without score returns: `list[Document]:`
5. Check source code: https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/vectorstores/base.py#L450
6. Document is a class defined in Documents package: https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/documents/base.py#L262
7. Document has an attribute `page_content` that gives the content
8. The vector store returns some non-mandatory attributes like `id` and a dict `metadata`
9. We want to pick the top result and use that to answer usser query  


In [4]:
vectorstore = InMemoryVectorStore.from_texts(
    text,
    embedding = embeddings
)

# results = vectorstore.similarity_search(query="premium car", k=1)
# for doc in results:
#     print(f"* {doc.page_content} [{doc.metadata}]")


# userQuery = 'In the rabbit story what was the name of the rabbit?'
userQuery = 'What was john guided by?'

results = vectorstore.similarity_search_with_score(query=userQuery, k=2)

for doc, score in results: 
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

selectedStory = doc.page_content


* [SIM=0.779186] In a lush green meadow, a curious rabbit named Hoppy ventured out on a sunny morning. With a twinkle in his eye, he hopped towards a mysterious forest, intrigued by the unknown.
    As he ventured deeper, Hoppy discovered a magical world filled with colorful flowers and friendly creatures. He made new friends, from wise old owls to playful squirrels.
    His adventure taught him about courage, friendship, and the beauty of nature. With a heart full of joy, Hoppy returned home, sharing his tales of wonder with his fellow rabbits, inspiring them to explore the world beyond their burrows. John, a blind man, navigated the world with an unwavering spirit. Despite his physical challenge, he embraced life with open arms. Guided by his trusty cane and the warmth of his smile, he ventured into the city, a place of endless possibilities.
    As he walked, he felt the energy of the bustling streets, the laughter of children, and the melody of street musicians. John's heart was fi

### Generating Answer from context RAG
1. Pass the story to the model along with 

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

sysMessage = SystemMessage('''Role: You are question answer bot. You need to refer to the story below and answer the users question
                           in an concise manner.''')
sysMessage1 = SystemMessage(f'Store: {selectedStory}')


humanMessage = HumanMessage(userQuery)

response = model.invoke([sysMessage, sysMessage1, humanMessage])
print(response)

content='There is no John mentioned in the story. The main character is Emily, and she was guided by her spontaneous decision to embark on a solo adventure.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 167, 'total_tokens': 198, 'completion_time': 0.025833333, 'prompt_time': 0.007453104, 'queue_time': 0.006565223, 'total_time': 0.033286437}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None} id='run-7a545b4b-a6d9-40a5-945e-e9071a7c3217-0' usage_metadata={'input_tokens': 167, 'output_tokens': 31, 'total_tokens': 198}


In [6]:
from langchain_core.messages import SystemMessage

s = SystemMessage([{'system message':'This is system message.'}])
s.content

[{'system message': 'This is system message.'}]